The code requires FEniCSx---see the README for details.

In [ ]:
%load_ext autoreload
%autoreload 2
# add path to code
import sys
sys.path.insert(0, '../source')

In [ ]:
import matplotlib.pylab as pylab
import matplotlib.pyplot as plt
import numpy as np
from dolfinx.fem import Constant, Function, FunctionSpace
from dolfinx.mesh import create_interval
from misc import interp
from mpi4py import MPI
from params import eta, H0, nt, nz, phi0, w0, zeta
from petsc4py import PETSc
from solvers import solve
from ufl import FiniteElement, MixedElement

Define domain:

In [ ]:
# generate mesh, initial domain is has a scaled height of 1 
domain = create_interval(MPI.COMM_WORLD,nz,[0,1])

In [ ]:
P1 = FiniteElement('P',domain.ufl_cell(),1)     
element = P1*P1
V = FunctionSpace(domain,element)   
initial = Function(V)
initial.sub(0).interpolate(lambda x:-x[0])
initial.sub(1).interpolate(lambda x:phi0+0*x[0])

In [ ]:
# # Dirichlet condition: set compaction rate (velocity) at the top
bc_top = {'type': 'velocity', 'value': -1}

# # Neumann condition: set the load (stress) at the top
# bc_top = {'type': 'stress', 'value': 1e1*H0/(w0*((4./3.)*eta + zeta))}

In [ ]:
w,phi,z = solve(domain,initial,bc_top)

Plot the solution:

In [ ]:
# time array for plotting
t = np.outer(np.linspace(0,1,nt),np.ones(nz))

plt.figure(figsize=(6,4))
plt.subplot(121)
plt.plot(t[:,0],z[:,-1],'--',color='crimson',linewidth=2)
plt.contourf(t,z,-w/np.max(np.abs(w)),cmap='Blues',vmin=0,vmax=1,levels=np.linspace(0,1,100),extend='both')
plt.ylabel(r'$z$',fontsize=18)
plt.xlabel(r'$t$',fontsize=18)
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)
cbar = plt.colorbar(orientation='horizontal',pad=0.2,ticks=np.linspace(0,1,5))
cbar.set_label(r'$w\,/\,w_0$',fontsize=24)
cbar.ax.tick_params(labelsize=12)
cbar.ax.set_xticklabels([0,0.25,0.5,0.75,1])

plt.subplot(122)
plt.plot(t[:,0],z[:,-1],'--',color='crimson',linewidth=2)
plt.contourf(t,z,phi/phi0,cmap='Blues',levels=np.linspace(0,1,100),vmin=0,vmax=1,extend='both')
plt.ylabel(r'$z$',fontsize=18)
plt.xlabel(r'$t$',fontsize=18)
plt.xticks(fontsize=12)
plt.ticklabel_format(useMathText=True)
plt.gca().yaxis.set_label_position("right")
plt.gca().yaxis.tick_right()
plt.yticks(fontsize=12)
cbar2 = plt.colorbar(orientation='horizontal',pad=0.2,ticks=np.linspace(0,1,5))
cbar2.set_label(r'$\phi\,/\,\phi_0$',fontsize=24)
cbar2.ax.tick_params(labelsize=12)
cbar2.ax.set_xticklabels([0,0.25,0.5,0.75,1])
plt.tight_layout()
plt.show()
plt.close()